In [2]:
import sqlite3 
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
# from lib import MongoHandler, WeatherGetter
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

In [3]:
cur.execute("""SELECT DISTINCT Date, HomeTeam, FTHG, AwayTeam, FTAG, FTR, match_id
                FROM teams_in_matches
                JOIN unique_teams
                USING(unique_team_id)
                JOIN matches
                USING (match_id)
                WHERE season IN (SELECT season
                                    FROM matches 
                                    WHERE season == '2011')
               ORDER BY ftr
                """)

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df = df.sort_values(by=['FTR'])
df.head()

,Date,HomeTeam,FTHG,AwayTeam,FTAG,FTR,Match_ID
0,2012-03-31,Nurnberg,0,Bayern Munich,1,A,1092
177,2011-09-17,Bolton,1,Norwich,2,A,44536
178,2011-09-17,Wolves,0,QPR,3,A,44539
179,2011-09-24,Wigan,1,Tottenham,2,A,44551
180,2011-10-01,Blackburn,0,Man City,4,A,44555


In [4]:
#AWAY WINS

cur.execute("""SELECT DISTINCT match_id, AwayTeam 
                FROM teams_in_matches
                JOIN unique_teams
                USING(unique_team_id)
                JOIN matches
                USING (match_id)
                WHERE season IN (SELECT season
                                    FROM matches 
                                    WHERE season == '2011')
               AND
               ftr = 'A'   
            """)

away_df = pd.DataFrame(cur.fetchall())
away_df.columns = [i[0] for i in cur.description]
away_df = away_df.groupby(away_df['AwayTeam']).count()

In [5]:
#HOME WINS
cur.execute("""SELECT DISTINCT match_id, HomeTeam 
                FROM teams_in_matches
                JOIN unique_teams
                USING(unique_team_id)
                JOIN matches
                USING (match_id)
                WHERE season IN (SELECT season
                                    FROM matches 
                                    WHERE season == '2011')
               AND
               ftr = 'H'   
            """)

home_df = pd.DataFrame(cur.fetchall())
home_df.columns = [i[0] for i in cur.description]
home_df = home_df.groupby(home_df['HomeTeam']).count()

In [6]:
#TOTAL WINS

df_wins = home_df.join(away_df, lsuffix='_Home', rsuffix='_Away')
df_wins

total_wins = []
home_wins = df_wins['Match_ID_Home']
away_wins = df_wins['Match_ID_Away']


for item in df_wins.index:
    wins = home_wins[item] + away_wins[item]
    total_wins.append(wins)

df_wins['Total_Wins'] = total_wins
df_wins.head()

,Match_ID_Home,Match_ID_Away,Total_Wins
HomeTeam,,,
Aachen,4,2,6
Arsenal,12,9,21
Aston Villa,4,3,7
Augsburg,6,2,8
Bayern Munich,14,9,23


In [7]:
#DRAWS
cur.execute("""SELECT DISTINCT match_id, HomeTeam, FTHG, AwayTeam, FTAG
                FROM teams_in_matches
                JOIN unique_teams
                USING(unique_team_id)
                JOIN matches
                USING (match_id)
                WHERE season IN (SELECT season
                                    FROM matches 
                                    WHERE season == '2011')
               AND
               ftr = 'D'   
            """)

draws_df = pd.DataFrame(cur.fetchall())
draws_df.columns = [i[0] for i in cur.description]
draws_df = draws_df.groupby(draws_df['AwayTeam'])
draws_df.head()

# home = draws_df['HomeTeam'].count()
# home
away = draws_df['AwayTeam'].count()
df_wins['Draws'] = away

In [8]:
#Home Losses
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

q ="""SELECT COUNT(HomeTeam), HomeTeam
        FROM df
        WHERE FTR = 'A'
        GROUP BY HomeTeam       
    """

results = pysqldf(q)
df3 = pd.DataFrame(results)

In [9]:
#Away Losses
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

q ="""SELECT COUNT(AwayTeam), AwayTeam
        FROM df
        WHERE FTR = 'H'
        GROUP BY AwayTeam        
    """

results = pysqldf(q)
df4 = pd.DataFrame(results)

In [10]:
df_losses = home_df.join(away_df, lsuffix='_Home', rsuffix='_Away')
df_losses.columns = ['Home_Losses', 'Away_Losses']

In [11]:
#TOTAL LOSSES
total_losses = []
home_losses = df_losses['Home_Losses']
away_losses = df_losses['Away_Losses']


for item in df_losses.index:
    loss = home_losses[item] + away_losses[item]
    total_losses.append(loss)

df_losses['Total_losses'] = total_losses
df_wins['Total_Losses'] = df_losses['Total_losses']

In [12]:
df_wins.head()

,Match_ID_Home,Match_ID_Away,Total_Wins,Draws,Total_Losses
HomeTeam,,,,,
Aachen,4,2,6,7,6
Arsenal,12,9,21,3,21
Aston Villa,4,3,7,10,7
Augsburg,6,2,8,7,8
Bayern Munich,14,9,23,3,23


In [14]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

q ="""SELECT Match_Id, AwayTeam, Count(HomeTeam), HomeTeam
        FROM df
        GROUP BY HomeTeam
    """


results = pysqldf(q)

In [ ]:
ts.get('https://api.darksky.net/forecast/958870d7f05d6ce07081ed0698de9eca/13.404954,52.520008,{}?exclude=currently,flags,minutely,hourly,alerts'.format(Timestamp))
r.json()

In [10]:
import requests
import json

r = requests.get('https://api.darksky.net/forecast/958870d7f05d6ce07081ed0698de9eca/13.404954,52.520008,1572548834')
a = r.json()

In [11]:
a["currently"]["summary"]

'Clear'

In [ ]:
for stamp in Timestam

In [7]:
unique_dates=cur.execute("SELECT DISTINCT Date FROM Matches WHERE Season IS 2011;").fetchall()

listo=[]
for date in unique_dates:
   for datetime in date:
       listo.append(datetime + "T13:00:00")
listo

weather_dates={}
for date in listo:
    call_weather = requests.get(f"https://api.darksky.net/forecast/4b592733b07f4d93b5f8e9f7b1daf53f/52.5200,13.4050,{date}").json()
    weather_type=call_weather["currently"]["summary"]
    weather_dates[date]=weather_type

    
# and then did this to get rid of the Time in the date so we can match it back to our "matches" table:

weather_dates_day=[]
for date in weather_dates:
   weather_dates_day[date[:10]]=weather_dates[date]
weather_dates_day

KeyError: 'currently'

In [ ]:
weather_dates